#### Retrieve historical data covering the #EgayPH, #FalconPH, and #Habagat effects in Metro Manila

This Jupyter notebook gathers the historical data during the days when #EgayPH, #FalconPH, and the Southwest Monsoon had major impact in Metro Manila. The meteorological parameters gathered here are total rainfall, maximum temperature, maximum heat index, maximum wind speed, maximum wind gust, maximum solar radiation, and minimum (relative) pressure.

The produced CSV file will be used to visualize the data using Tableau Public.

##### Import necessary libraries

In [1]:
import pandas as pd
import psycopg2
import os
import datetime as dt
from sqlalchemy import create_engine
from dotenv import find_dotenv, load_dotenv

##### Formatting for DataFrame to show all columns

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

##### Get the necessary variables for the database connection

In [3]:
dotenv_path = find_dotenv()

load_dotenv(dotenv_path)

DB_HOST = os.getenv("POSTGRES_HOST")
DB_NAME = os.getenv("POSTGRES_DATABASE")
DB_USER = os.getenv("POSTGRES_USERNAME")
DB_PASS = os.getenv("POSTGRES_PASSWORD")

##### USER DEFINED VARIABLES HERE
The values for the datetime fields were filled like these as it was during this range that the tropical cyclones were within the Philippine Area of Responsibility.

In [4]:
YEAR_1 = 2023
MONTH_1 = 7
DAY_1 = 22
HOUR_1 = 0
MINUTE_1 = 0
SECOND_1 = 0

YEAR_2 = 2023
MONTH_2 = 8
DAY_2 = 3
HOUR_2 = 23
MINUTE_2 = 59
SECOND_2 = 59


start_date = str(dt.datetime(year=YEAR_1, month=MONTH_1, day=DAY_1, hour=HOUR_1, minute=MINUTE_1, second=SECOND_1))
end_date = str(dt.datetime(year=YEAR_2, month=MONTH_2, day=DAY_2, hour=HOUR_2, minute=MINUTE_2, second=SECOND_2))
start_date_file_name = str(dt.date(year=YEAR_1, month=MONTH_1, day=DAY_1))
end_date_file_name = str(dt.date(year=YEAR_2, month=MONTH_2, day=DAY_2))

start_date_file_name

'2023-07-22'

##### Fetch data regarding the stated dates from the database containing historical data

In [5]:
conn = psycopg2.connect(
    host = DB_HOST,
    database=DB_NAME,
    user=DB_USER,
    password=DB_PASS
)

cursor = conn.cursor()

# Define the table name
table_name = 'measurements'

# Check if the table exists
cursor.execute(
    """
    SELECT *
    FROM {0}
    WHERE obs_time_local >= %s AND obs_time_local <= %s
    """.format(table_name),
    (start_date, end_date)
)

rows = cursor.fetchall()

##### Store rows in a DataFrame

In [6]:
df = pd.DataFrame(rows, columns=[column[0] for column in cursor.description])
df = df.drop(labels=['id'], axis=1)
df = df.sort_values(by='obs_time_local')
df['obs_day'] = pd.to_datetime(df['obs_time_local'].dt.date)
df.tail(10)

,station_id,epoch,humidity_avg,humidity_high,humidity_low,obs_time_local,obs_time_utc,solar_radiation_high,uv_high,wind_direction_avg,dew_point_avg,dew_point_high,dew_point_low,heat_index_avg,heat_index_high,heat_index_low,precipitation_rate,precipitation_total,pressure_max,pressure_min,pressure_trend,qc_status,temperature_avg,temperature_high,temperature_low,wind_chill_avg,wind_chill_high,wind_chill_low,wind_gust_avg,wind_gust_high,wind_gust_low,wind_speed_avg,wind_speed_high,wind_speed_low,obs_day
21755,IPARAA10,1691078093,96.0,96.0,96.0,2023-08-03 23:54:53,2023-08-03 23:54:53+08:00,0.0,0.0,204.0,25.0,25.0,25.0,28.0,28.0,28.0,0.0,5.59,1014.90,1014.90,0.00,1,26.0,26.0,26.0,26.0,26.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-08-03
11199,IMETROMA22,1691078095,97.0,97.0,97.0,2023-08-03 23:54:55,2023-08-03 23:54:55+08:00,0.0,0.0,315.0,25.0,25.0,25.0,28.0,28.0,28.0,0.0,4.32,1012.53,1012.19,0.00,1,26.0,26.0,26.0,26.0,26.0,26.0,1.0,2.0,0.0,0.0,0.0,0.0,2023-08-03
3743,IBULACAN2,1691078096,99.0,99.0,99.0,2023-08-03 23:54:56,2023-08-03 23:54:56+08:00,0.0,0.0,96.0,27.0,27.0,27.0,32.0,32.0,32.0,0.0,11.18,1009.14,1008.81,-4.23,-1,27.0,27.0,27.0,27.0,27.0,27.0,4.0,6.0,0.0,2.0,5.0,0.0,2023-08-03
7402,IMAKAT1,1691078096,95.0,95.0,95.0,2023-08-03 23:54:56,2023-08-03 23:54:56+08:00,0.0,0.0,0.0,26.0,26.0,26.0,33.0,33.0,33.0,0.0,3.30,1013.07,1013.00,-0.82,1,27.0,27.0,27.0,27.0,27.0,27.0,1.0,2.0,0.0,0.0,0.0,0.0,2023-08-03
23611,IRIZBULA2,1691078371,92.0,92.0,92.0,2023-08-03 23:59:31,2023-08-03 23:59:31+08:00,0.0,0.0,191.0,25.0,25.0,25.0,28.0,28.0,28.0,0.0,13.49,1015.61,1015.24,3.85,1,26.0,26.0,26.0,26.0,26.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-08-03
3744,IBULACAN2,1691078384,99.0,99.0,99.0,2023-08-03 23:59:44,2023-08-03 23:59:44+08:00,0.0,0.0,97.0,27.0,27.0,27.0,32.0,32.0,32.0,0.0,11.18,1009.14,1008.81,0.00,-1,27.0,27.0,27.0,27.0,27.0,27.0,2.0,9.0,0.0,1.0,5.0,0.0,2023-08-03
16708,IMUNTI6,1691078392,92.0,93.0,92.0,2023-08-03 23:59:52,2023-08-03 23:59:52+08:00,0.0,0.0,242.0,26.0,26.0,26.0,31.0,32.0,31.0,0.0,1.78,1012.87,1012.87,0.00,1,27.0,27.0,27.0,27.0,27.0,27.0,4.0,8.0,3.0,4.0,7.0,3.0,2023-08-03
21756,IPARAA10,1691078393,96.0,96.0,96.0,2023-08-03 23:59:53,2023-08-03 23:59:53+08:00,0.0,0.0,204.0,25.0,25.0,25.0,28.0,28.0,28.0,0.0,5.59,1014.90,1014.90,0.00,1,26.0,26.0,26.0,26.0,26.0,26.0,1.0,2.0,0.0,1.0,2.0,0.0,2023-08-03
11200,IMETROMA22,1691078395,97.0,98.0,97.0,2023-08-03 23:59:55,2023-08-03 23:59:55+08:00,0.0,0.0,25.0,25.0,26.0,25.0,28.0,28.0,28.0,0.0,4.32,1012.53,1011.85,-4.13,1,26.0,26.0,26.0,26.0,26.0,26.0,2.0,2.0,0.0,0.0,0.0,0.0,2023-08-03
7403,IMAKAT1,1691078396,95.0,95.0,95.0,2023-08-03 23:59:56,2023-08-03 23:59:56+08:00,0.0,0.0,0.0,27.0,27.0,26.0,33.0,33.0,33.0,0.0,3.30,1013.00,1012.87,-1.65,1,27.0,27.0,27.0,27.0,27.0,27.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-08-03


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23623 entries, 18648 to 7403
Data columns (total 35 columns):
 #   Column                Non-Null Count  Dtype                    
---  ------                --------------  -----                    
 0   station_id            23623 non-null  object                   
 1   epoch                 23623 non-null  int64                    
 2   humidity_avg          23612 non-null  float64                  
 3   humidity_high         23612 non-null  float64                  
 4   humidity_low          23612 non-null  float64                  
 5   obs_time_local        23623 non-null  datetime64[ns]           
 6   obs_time_utc          23623 non-null  datetime64[ns, UTC+08:00]
 7   solar_radiation_high  23612 non-null  float64                  
 8   uv_high               23612 non-null  float64                  
 9   wind_direction_avg    23612 non-null  float64                  
 10  dew_point_avg         23611 non-null  float64               

##### Get the total amount of rainfall per day

In [8]:
daily_max_meteorological_params_df = df.groupby(['station_id', 'obs_day']).agg({
    'obs_time_local': 'last',
    'precipitation_total': 'max'
}).reset_index()

In [9]:
daily_max_meteorological_params_df = daily_max_meteorological_params_df.rename(columns={'precipitation_total': 'recorded_data'})
daily_max_meteorological_params_df['parameter'] = 'Total Rainfall'
daily_max_meteorological_params_df.head()

,station_id,obs_day,obs_time_local,recorded_data,parameter
0,IBULACAN2,2023-07-22,2023-07-22 23:59:34,13.21,Total Rainfall
1,IBULACAN2,2023-07-23,2023-07-23 23:59:18,144.27,Total Rainfall
2,IBULACAN2,2023-07-24,2023-07-24 23:59:47,1.78,Total Rainfall
3,IBULACAN2,2023-07-25,2023-07-25 23:59:45,10.41,Total Rainfall
4,IBULACAN2,2023-07-26,2023-07-26 23:59:59,69.09,Total Rainfall


##### Get the maximum temperature per day and its observation time

In [10]:
max_temp_df = df.groupby(['station_id', 'obs_day']).apply(lambda x: x.loc[x['temperature_avg'].idxmax()]).reset_index(drop=True)[['station_id', 'obs_day', 'obs_time_local', 'temperature_avg']]
max_temp_df['parameter'] = 'Maximum Temperature'
max_temp_df = max_temp_df.rename(columns={'temperature_avg': 'recorded_data'})
max_temp_df.head(10)

,station_id,obs_day,obs_time_local,recorded_data,parameter
0,IBULACAN2,2023-07-22,2023-07-22 10:39:51,32.0,Maximum Temperature
1,IBULACAN2,2023-07-23,2023-07-23 11:44:54,32.0,Maximum Temperature
2,IBULACAN2,2023-07-24,2023-07-24 13:14:50,33.0,Maximum Temperature
3,IBULACAN2,2023-07-25,2023-07-25 10:19:54,32.0,Maximum Temperature
4,IBULACAN2,2023-07-26,2023-07-26 10:09:52,30.0,Maximum Temperature
5,IBULACAN2,2023-07-27,2023-07-27 02:44:47,28.0,Maximum Temperature
6,IBULACAN2,2023-07-28,2023-07-28 10:24:29,28.0,Maximum Temperature
7,IBULACAN2,2023-07-29,2023-07-29 11:24:59,30.0,Maximum Temperature
8,IBULACAN2,2023-07-30,2023-07-30 01:29:47,50.0,Maximum Temperature
9,IBULACAN2,2023-07-31,2023-07-31 11:44:54,31.0,Maximum Temperature


In [11]:
daily_max_meteorological_params_df = pd.concat([daily_max_meteorological_params_df, max_temp_df], ignore_index=True)

In [12]:
daily_max_meteorological_params_df.head(10)

,station_id,obs_day,obs_time_local,recorded_data,parameter
0,IBULACAN2,2023-07-22,2023-07-22 23:59:34,13.21,Total Rainfall
1,IBULACAN2,2023-07-23,2023-07-23 23:59:18,144.27,Total Rainfall
2,IBULACAN2,2023-07-24,2023-07-24 23:59:47,1.78,Total Rainfall
3,IBULACAN2,2023-07-25,2023-07-25 23:59:45,10.41,Total Rainfall
4,IBULACAN2,2023-07-26,2023-07-26 23:59:59,69.09,Total Rainfall
5,IBULACAN2,2023-07-27,2023-07-27 23:59:25,278.89,Total Rainfall
6,IBULACAN2,2023-07-28,2023-07-28 23:59:56,206.50,Total Rainfall
7,IBULACAN2,2023-07-29,2023-07-29 23:59:54,175.51,Total Rainfall
8,IBULACAN2,2023-07-30,2023-07-30 23:59:51,25.40,Total Rainfall
9,IBULACAN2,2023-07-31,2023-07-31 23:59:49,32.00,Total Rainfall


##### Get the maximum heat index per day and its observation time

In [13]:
max_heat_index_df = df.groupby(['station_id', 'obs_day']).apply(lambda x: x.loc[x['heat_index_avg'].idxmax()]).reset_index(drop=True)[['station_id', 'obs_day', 'obs_time_local', 'heat_index_avg']]
max_heat_index_df['parameter'] = 'Maximum Heat Index'
max_heat_index_df = max_heat_index_df.rename(columns={'heat_index_avg': 'recorded_data'})

In [14]:
daily_max_meteorological_params_df = pd.concat([daily_max_meteorological_params_df, max_heat_index_df], ignore_index=True)

In [15]:
daily_max_meteorological_params_df.tail(10)

,station_id,obs_day,obs_time_local,recorded_data,parameter
263,IRIZBULA2,2023-07-25,2023-07-25 11:29:52,45.0,Maximum Heat Index
264,IRIZBULA2,2023-07-26,2023-07-26 10:34:56,39.0,Maximum Heat Index
265,IRIZBULA2,2023-07-27,2023-07-27 10:59:58,39.0,Maximum Heat Index
266,IRIZBULA2,2023-07-28,2023-07-28 09:29:50,39.0,Maximum Heat Index
267,IRIZBULA2,2023-07-29,2023-07-29 11:39:56,44.0,Maximum Heat Index
268,IRIZBULA2,2023-07-30,2023-07-30 12:34:51,36.0,Maximum Heat Index
269,IRIZBULA2,2023-07-31,2023-07-31 11:14:49,41.0,Maximum Heat Index
270,IRIZBULA2,2023-08-01,2023-08-01 16:54:47,33.0,Maximum Heat Index
271,IRIZBULA2,2023-08-02,2023-08-02 11:14:47,38.0,Maximum Heat Index
272,IRIZBULA2,2023-08-03,2023-08-03 13:34:36,45.0,Maximum Heat Index


##### Get the maximum wind speed per day and its observation time

In [16]:
max_wind_speed_df = df.groupby(['station_id', 'obs_day']).apply(lambda x: x.loc[x['wind_speed_avg'].idxmax()]).reset_index(drop=True)[['station_id', 'obs_day', 'obs_time_local', 'wind_speed_avg']]
max_wind_speed_df['parameter'] = 'Maximum Wind Speed'
max_wind_speed_df = max_wind_speed_df.rename(columns={'wind_speed_avg': 'recorded_data'})

In [17]:
daily_max_meteorological_params_df = pd.concat([daily_max_meteorological_params_df, max_wind_speed_df], ignore_index=True)

In [18]:
daily_max_meteorological_params_df.tail(10)

,station_id,obs_day,obs_time_local,recorded_data,parameter
354,IRIZBULA2,2023-07-25,2023-07-25 15:59:44,13.0,Maximum Wind Speed
355,IRIZBULA2,2023-07-26,2023-07-26 01:14:56,16.0,Maximum Wind Speed
356,IRIZBULA2,2023-07-27,2023-07-27 14:19:58,11.0,Maximum Wind Speed
357,IRIZBULA2,2023-07-28,2023-07-28 11:49:49,11.0,Maximum Wind Speed
358,IRIZBULA2,2023-07-29,2023-07-29 16:59:55,11.0,Maximum Wind Speed
359,IRIZBULA2,2023-07-30,2023-07-30 11:34:51,7.0,Maximum Wind Speed
360,IRIZBULA2,2023-07-31,2023-07-31 12:39:53,9.0,Maximum Wind Speed
361,IRIZBULA2,2023-08-01,2023-08-01 11:09:44,6.0,Maximum Wind Speed
362,IRIZBULA2,2023-08-02,2023-08-02 11:19:51,9.0,Maximum Wind Speed
363,IRIZBULA2,2023-08-03,2023-08-03 13:24:54,12.0,Maximum Wind Speed


##### Get the maximum wind gust per day and its observation time

In [19]:
max_wind_gust_df = df.groupby(['station_id', 'obs_day']).apply(lambda x: x.loc[x['wind_gust_avg'].idxmax()]).reset_index(drop=True)[['station_id', 'obs_day', 'obs_time_local', 'wind_gust_avg']]
max_wind_gust_df['parameter'] = 'Maximum Wind Gust'
max_wind_gust_df = max_wind_gust_df.rename(columns={'wind_gust_avg': 'recorded_data'})

In [20]:
daily_max_meteorological_params_df = pd.concat([daily_max_meteorological_params_df, max_wind_gust_df], ignore_index=True)

In [21]:
daily_max_meteorological_params_df.tail(10)

,station_id,obs_day,obs_time_local,recorded_data,parameter
445,IRIZBULA2,2023-07-25,2023-07-25 15:59:44,18.0,Maximum Wind Gust
446,IRIZBULA2,2023-07-26,2023-07-26 01:14:56,22.0,Maximum Wind Gust
447,IRIZBULA2,2023-07-27,2023-07-27 14:19:58,16.0,Maximum Wind Gust
448,IRIZBULA2,2023-07-28,2023-07-28 11:49:49,18.0,Maximum Wind Gust
449,IRIZBULA2,2023-07-29,2023-07-29 11:44:44,15.0,Maximum Wind Gust
450,IRIZBULA2,2023-07-30,2023-07-30 12:29:47,11.0,Maximum Wind Gust
451,IRIZBULA2,2023-07-31,2023-07-31 12:39:53,13.0,Maximum Wind Gust
452,IRIZBULA2,2023-08-01,2023-08-01 06:54:48,8.0,Maximum Wind Gust
453,IRIZBULA2,2023-08-02,2023-08-02 11:19:51,12.0,Maximum Wind Gust
454,IRIZBULA2,2023-08-03,2023-08-03 13:24:54,17.0,Maximum Wind Gust


##### Get the maximum solar radiation per day and its observation time

In [22]:
max_solar_radiation_df = df.groupby(['station_id', 'obs_day']).apply(lambda x: x.loc[x['solar_radiation_high'].idxmax()]).reset_index(drop=True)[['station_id', 'obs_day', 'obs_time_local', 'solar_radiation_high']]
max_solar_radiation_df['parameter'] = 'Maximum Solar Radiation'
max_solar_radiation_df = max_solar_radiation_df.rename(columns={'solar_radiation_high': 'recorded_data'})

In [23]:
daily_max_meteorological_params_df = pd.concat([daily_max_meteorological_params_df, max_solar_radiation_df], ignore_index=True)

In [24]:
daily_max_meteorological_params_df.tail(10)

,station_id,obs_day,obs_time_local,recorded_data,parameter
536,IRIZBULA2,2023-07-25,2023-07-25 11:19:44,775.61,Maximum Solar Radiation
537,IRIZBULA2,2023-07-26,2023-07-26 12:24:48,432.45,Maximum Solar Radiation
538,IRIZBULA2,2023-07-27,2023-07-27 10:54:38,645.45,Maximum Solar Radiation
539,IRIZBULA2,2023-07-28,2023-07-28 11:14:53,764.72,Maximum Solar Radiation
540,IRIZBULA2,2023-07-29,2023-07-29 11:04:44,828.95,Maximum Solar Radiation
541,IRIZBULA2,2023-07-30,2023-07-30 12:09:47,769.98,Maximum Solar Radiation
542,IRIZBULA2,2023-07-31,2023-07-31 09:54:49,652.42,Maximum Solar Radiation
543,IRIZBULA2,2023-08-01,2023-08-01 13:24:56,238.92,Maximum Solar Radiation
544,IRIZBULA2,2023-08-02,2023-08-02 10:44:39,396.86,Maximum Solar Radiation
545,IRIZBULA2,2023-08-03,2023-08-03 12:49:32,927.43,Maximum Solar Radiation


In [25]:
daily_max_meteorological_params_df.to_csv('daily_max_meteorological_params_' + start_date_file_name + '_' + end_date_file_name + '.csv', 
                                          index=False, 
                                          header=True, 
                                          encoding='utf-8')

##### Get the minimum pressure per day and its observation time

In [26]:
min_pressure_df = df.groupby(['station_id', 'obs_day']).apply(lambda x: x.loc[x['pressure_min'].idxmax()]).reset_index(drop=True)[['station_id', 'obs_day', 'obs_time_local', 'pressure_min']]
min_pressure_df['parameter'] = 'Minimum Pressure'
min_pressure_df = min_pressure_df.rename(columns={'pressure_min': 'recorded_data'})

In [27]:
daily_max_meteorological_params_df = pd.concat([daily_max_meteorological_params_df, min_pressure_df], ignore_index=True)

In [28]:
daily_max_meteorological_params_df.tail(10)

,station_id,obs_day,obs_time_local,recorded_data,parameter
627,IRIZBULA2,2023-07-25,2023-07-25 00:04:49,1004.03,Minimum Pressure
628,IRIZBULA2,2023-07-26,2023-07-26 21:59:59,1005.22,Minimum Pressure
629,IRIZBULA2,2023-07-27,2023-07-27 23:29:49,1007.92,Minimum Pressure
630,IRIZBULA2,2023-07-28,2023-07-28 21:24:45,1009.92,Minimum Pressure
631,IRIZBULA2,2023-07-29,2023-07-29 21:24:59,1010.33,Minimum Pressure
632,IRIZBULA2,2023-07-30,2023-07-30 22:19:54,1010.33,Minimum Pressure
633,IRIZBULA2,2023-07-31,2023-07-31 08:54:50,1009.62,Minimum Pressure
634,IRIZBULA2,2023-08-01,2023-08-01 21:24:55,1011.62,Minimum Pressure
635,IRIZBULA2,2023-08-02,2023-08-02 22:14:47,1013.34,Minimum Pressure
636,IRIZBULA2,2023-08-03,2023-08-03 22:24:54,1015.92,Minimum Pressure
